# Synthetic Polynomial Dataset Dimension Estimation

Autoencoder innermost layer is refashioned into singular value proxies (SVP).  These SVP are used to estimate dimension of the dataset.

###### What is Synthetic Polynomial? 

We begin out experiments by considering synthetic polynomial $f(x,y,z)$, where $f(x,y,z)$ comprises $3^{rd}$ degree, $2^{nd}$ degree, $1^{st}$ terms and cross terms of $x, y, $ and $z$.

\begin{equation}
  \begin{aligned}
    f(x,y,z) &= c_0x^3 + c_1x^2y + c_2x^2z + c_3x^2 + c_4xy^2 \\ 
     &\qquad + c_5xyz + c_6xy + c_7xz^2 + c_8xz + c_9x\\
     &\qquad + c_{10}y^3 + c_{11}y^2z + c_{12}y^2 + c_{13}yz^2 + c_{14}yz\\ 
     &\qquad + c_{15}y + c_{16}z^3 + c_{17}z^2 + c_{18}z + c_{k}
  \end{aligned}
\end{equation}

The coefficients $c_0, c_1, c_2, c_3, c_4, ..., c_{17}, c_{18}$, and independent variables $x, y, z$ are randomly generated. 

Random numbers are generated from $c_k \sim \mathcal{N} (\mu=0,\sigma^{2}=1.0)$, where $k \in [0, 18]$ and $-1 < c_k < 1$. 

We generate $5000$ tuples of $x, y, z$, a tuple for each row, and $784$ tuples of $c_0, c_1, c_2, c_3, c_4, ..., c_{17}, c_{18}$, a tuple for each column. $f(x,y,z)$ non-linearly maps the $3$ dimensional data to $784$ dimensions. Because the coefficients are randomly generated, we call $f(x,y,z)$ <i>randomly generated taylor polynomial</i>. 

We $DE(\boldsymbol{X})$, where $\boldsymbol{X} \in\mathbb {R}^{5000 \times 784}$. 

The first $3000$ instances of $\boldsymbol{X}$ forms the training set and the remaining $2000$ instances form the test set.

In [183]:
import numpy as np
import pandas as pd
import os
import sys
import csv
import json

from datetime import datetime

from tensorflow.keras.layers import Input, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
from keras import regularizers
from keras.callbacks import Callback
from keras import backend as K
import tensorflow as tf
from keras.regularizers import Regularizer

import scipy.sparse
from keras.models import load_model

The iPython notebook code is only being provided for convenience.  

All Linux code along with scripts is available at https://github.com/nitishbahadur/book_chapter. Our Linux code is based on tensorflow 1.x.  Python package requirements were exported and available https://github.com/nitishbahadur/book_chapter/blob/master/src/requirements.txt.

We run our production code on https://arc.wpi.edu/cluster-documentation/build/html/clusters.html for performance reasons.

In [184]:
tf.compat.v1.get_default_graph()
tf.compat.v1.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

The synthetic polynomial data is loaded from data folder. We treat the first $3000$ instances as training dataset and remaining $2000$ instances as test dataset.

In [185]:
base_input_folder = r'../data/synthetic_polynomial/input'
base_output_folder = r'../data/synthetic_polynomial/output'
input_filename = r'Y_{}.csv'

In [186]:
def get_input_filename(std_dev):
    filename = input_filename.format(std_dev)
    filepath = os.path.join(base_input_folder, filename)
    return filepath

def get_synthetic_data(std_dev):
    filepath = get_input_filename(std_dev)
    print(f"get_synthetic_data is loading {filepath}")
    df = pd.read_csv(filepath, header=None)
    X = df.values
    X = X.astype('float32')
    X = X / (np.max(X) - np.min(X))
    x_train = X[0:3000,:]
    x_test = X[3000:,:]
    return x_train, x_test

Build the autoencoder model where the innermost layer is using a sigmoid activation function.  The autoencoder also uses dropout layers to control for overfitting.  We use a custom loss function.

In [187]:
def build_lite_ae_model(l1_reg, encoding_dim, layer1_dropout, layer2_dropout, layer3_dropout):
    input_img = Input(shape=(784,))
    encoded = Dense(392, activation='relu')(input_img)
    encoded = Dropout(layer1_dropout)(encoded)
    encoded = Dense(128, activation='relu')(encoded)
    encoded = Dropout(layer2_dropout)(encoded)

    # an additional layer with 30% dropout added to encoder
#     encoded = Dense(64, activation='relu')(encoded)
#     encoded = Dropout(layer3_dropout)(encoded)

    z_layer_input = Lambda(lambda  x: K.l2_normalize(x,axis=1))(encoded)
    encoded = Dense(encoding_dim, activation='sigmoid')(z_layer_input)
    encoded_norm = Lambda(lambda  x: K.l2_normalize(x,axis=1))(encoded)
    
    # an additional layer to match the encoder is added to decoder
#     decoded = Dense(64, activation='relu')(encoded)

    # was in the original ICMLA AEDE
    decoded = Dense(128, activation='relu')(encoded)
#     decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(392, activation='relu')(decoded)
    decoded = Dense(784, activation='tanh')(decoded)

    # create autoencoder
    autoencoder = Model(input_img, decoded)

    # create encoder
    encoder = Model(input_img, encoded)

    # create decoder model
    encoded_input = Input(shape=(encoding_dim,))
#     deco = autoencoder.layers[-4](encoded_input) # new code to match additional 64 layer encoder/decoder
    deco = autoencoder.layers[-3](encoded_input) # new code to match additional 64 layer encoder/decoder
#     deco = autoencoder.layers[-3](deco)
    deco = autoencoder.layers[-2](deco)
    deco = autoencoder.layers[-1](deco)
    decoder = Model(encoded_input, deco)    

    # autoencoder.compile(optimizer='adadelta', loss='mse') 
    autoencoder.compile(optimizer='adadelta', loss=mse_regularized_loss(encoded_norm, l1_reg)) 
    return encoder, decoder, autoencoder


def mse_regularized_loss(encoded_layer, lambda_):    
    def loss(y_true, y_pred):
        return K.mean(K.square(y_pred - y_true) + lambda_ * K.sum(K.abs(encoded_layer)))
    return loss 

The utility functions provided below is equivalent to the python code we use on HPC cluster.  We provide this for completeness here.

In [188]:
def get_output_model_path(output_file_type, encoding_dim, l1_reg):
    filename = r'{}_l1_reg_{}_{}.h5'.format(output_file_type, encoding_dim, l1_reg)
    filepath = os.path.join(base_output_folder, filename)
    return filepath

def save_model(encoding_dim, l1_reg, autoencoder, encoder, decoder):
    autoencoder_model_path = get_output_model_path("autoencoder", encoding_dim, l1_reg)
    encoder_model_path = get_output_model_path("encoder", encoding_dim, l1_reg)
    decoder_model_path = get_output_model_path("decoder", encoding_dim, l1_reg)

    autoencoder.save(autoencoder_model_path)
    print("autoencoder saved!!!")

    encoder.save(encoder_model_path) 
    print("encoder saved!!!")

    decoder.save(decoder_model_path) 
    print("decoder saved!!!")

def save_history(encoding_dim, l1_reg, history):
    history_filename = get_output_model_path("history", encoding_dim, l1_reg)
    with open(history_filename, 'w') as f:
        json.dump(history.history, f)

def save_intermediate_training(x, encoder, decoder, epoch):
    input_type = 'train'
    x_encoded = encoder.predict(x)
    x_reconstructed = decoder.predict(x_encoded)

    x_encoded_filename = r"../data/synthetic_polynomial/output/x_{}_{}_encoded_{}_{}"
    np.save(x_encoded_filename.format(input_type, epoch, encoding_dim, l1_reg), x_encoded)

    x_reconstructed_filename = r"../data/synthetic_polynomial/output/x_{}_{}_reconstructed_{}_{}"
    np.save(x_reconstructed_filename.format(input_type, epoch, encoding_dim, l1_reg), x_reconstructed)


def save_output(x, autoencoder, encoder, decoder, layer1_dropout, layer2_dropout, layer3_dropout, input_type):
    print("{} Original : ".format(input_type))
    print(x)

    print("{} Predicted : ".format(input_type))
    x_predicted = autoencoder.predict(x)
    print(x_predicted)

    print("{} Original->Encoded->Decoded(Reconsturcted) : ".format(input_type))
    x_encoded = encoder.predict(x)
    x_reconstructed = decoder.predict(x_encoded)
    print(x_reconstructed)

    print("{} Encoded : ".format(input_type))
    print(x_encoded)    

    x_filename = r"../data/synthetic_polynomial/output/x_{}_{}_{}_{}_{}_{}"
    np.save(x_filename.format(input_type, encoding_dim, l1_reg, layer1_dropout, layer2_dropout, layer3_dropout), x)

    x_encoded_filename = r"../data/synthetic_polynomial/output/x_{}_encoded_{}_{}_{}_{}_{}"
    np.save(x_encoded_filename.format(input_type, encoding_dim, l1_reg, layer1_dropout, layer2_dropout, layer3_dropout), x_encoded)

    x_predicted_filename = r"../data/synthetic_polynomial/output/x_{}_predicted_{}_{}_{}_{}_{}"
    np.save(x_predicted_filename.format(input_type, encoding_dim, l1_reg, layer1_dropout, layer2_dropout, layer3_dropout), x_predicted)

class SaveIntermediateTrainingOutput(Callback):
    def __init__(self, x, encoder, decoder):
        super(Callback, self).__init__()
        self.x = x
        self.encoder = encoder
        self.decoder = decoder
        self.counter = 1

    def on_epoch_end(self, epoch, logs={}):
        if epoch % 100 == 0:
            print("File counter: {}".format(self.counter*(epoch+1)))
            save_intermediate_training(self.x, self.encoder, self.decoder, self.counter*(epoch+1))
            self.counter = self.counter + 1

Estimate dimension by counting how many singular value proxies are greater than 1%

In [189]:
def count_gt_threshold(z, threshold):
    tot = sum(z)
    z_pct = [(i/tot) for i in sorted(z, reverse=True)]
    z_gt_theta = [i for i in z_pct if i >= threshold]
    return len(z_gt_theta)

def sort_by_row(z):
    z_sorted = None
    for i in np.arange(z.shape[0]):
        z_s = sorted(z[i,:], reverse=True)
        if z_sorted is None:
            z_sorted = z_s
        else:
            z_sorted = np.vstack((z_sorted,z_s))
    return z_sorted

For convenience we provide default values from run_polynomial_de.sh script.  The script is used to run DE process on High Performance Computing cluster at WPI

In [190]:
# sparsity parameter
l1_reg = 3e-2

# number of nodes in innermost hidden layer
encoding_dim = 16

# number of times you want to run 100 epochs
# DE converges slowly.
num_epochs = 40

# the batch size
batch_size = 64

# 30% of nodes are dropped out
layer1_dropout = 0.3

# 30% of nodes are dropped out
layer2_dropout = 0.3

# 30% of nodes are dropped out
layer3_dropout = 0.3

# the input file type to be loaded
std_dev = 1.0

Load synthetic data

In [191]:
x_train, x_test = get_synthetic_data(std_dev)

get_synthetic_data is loading ../data/synthetic_polynomial/input\Y_1.0.csv


In [192]:
print("Running standard AE with the following parameters : ")
print("x_train dimension : ({} x {})".format(x_train.shape[0], x_train.shape[1]))
print("encoding_dim dimension : {}".format(encoding_dim))
print("epochs : {} batch_size : {}".format(num_epochs*100, batch_size))
print("layer1_dropout : {} layer2_dropout : {} layer3_dropout : {}".format(layer1_dropout, layer2_dropout, layer3_dropout))
print("std_dev : {}".format(std_dev))
print("Running encoding_dim: {} l1_reg: {}".format(encoding_dim, l1_reg))

Running standard AE with the following parameters : 
x_train dimension : (3000 x 784)
encoding_dim dimension : 16
epochs : 4000 batch_size : 64
layer1_dropout : 0.3 layer2_dropout : 0.3 layer3_dropout : 0.3
std_dev : 1.0
Running encoding_dim: 16 l1_reg: 0.03


Create the encoder, decoder, and autoencoder model

In [193]:
encoder, decoder, autoencoder = build_lite_ae_model(l1_reg, encoding_dim, layer1_dropout, layer2_dropout, layer3_dropout)

In [194]:
svp_dict_ = {}
dim_dict_ = {}
for i in range(1, num_epochs+1):
        history = autoencoder.fit(x_train, x_train, epochs=100, batch_size=batch_size, verbose=0)
        z = encoder.predict(x_test) # use x_test
        z_row_sorted = sort_by_row(z)
        z_mu = np.mean(z_row_sorted, axis=0)
        gte_sorted = count_gt_threshold(z_mu, 0.01)
        
        z_mu_1 = sorted(np.mean(z, axis=0), reverse=True)
        gte_dim = count_gt_threshold(z_mu_1, 0.01)
        loss = history.history['loss'][-1]
                
        print("AE,{},{},{:.4f},{},{}".format(std_dev, i*100, loss, gte_sorted, gte_dim))
        
        converted_list = [str(np.round(element, 4)) for element in z_mu_1]
        svps = ",".join(converted_list)    
        print(svps)
        print()
        
        # save it for plotting later
        svp_dict_[i*100] = svps
        dim_dict_[i*100] = gte_sorted

AE,1.0,100,7.5851,16,16
0.6118,0.5994,0.5948,0.5804,0.5595,0.5294,0.5235,0.5217,0.5112,0.4635,0.462,0.4135,0.4024,0.3937,0.375,0.3371

AE,1.0,200,7.4694,16,16
0.6558,0.6527,0.6487,0.6392,0.5935,0.5612,0.5342,0.5195,0.5133,0.4155,0.4086,0.385,0.3304,0.3202,0.3049,0.2879

AE,1.0,300,7.2714,16,16
0.7095,0.7041,0.6981,0.6957,0.6326,0.596,0.538,0.5119,0.5002,0.3481,0.3355,0.3295,0.2726,0.2403,0.2359,0.2254

AE,1.0,400,6.9928,16,16
0.7632,0.7542,0.7506,0.742,0.6757,0.6334,0.5229,0.4792,0.4636,0.265,0.2631,0.2497,0.214,0.1868,0.1721,0.1608

AE,1.0,500,6.6665,16,16
0.8073,0.8012,0.7904,0.7821,0.7191,0.6701,0.4734,0.4076,0.3889,0.2003,0.1885,0.1746,0.1641,0.146,0.1225,0.1147

AE,1.0,600,6.3144,16,16
0.84,0.8381,0.8221,0.8145,0.7577,0.7027,0.3788,0.3035,0.2846,0.1507,0.1332,0.1253,0.1224,0.1141,0.0896,0.084

AE,1.0,700,5.9607,16,16
0.8646,0.8644,0.8466,0.8395,0.7886,0.7283,0.2607,0.2039,0.1901,0.114,0.0967,0.0965,0.0901,0.0883,0.0677,0.0636

AE,1.0,800,5.6627,14,14
0.8832,0.8827,0.8654,0.8579,0.

The singular value proxy keeps reducing.  Notice the values on the left are increasing and the values on the right are decreasing.  We take a snapshot every 100 epochs.